In [57]:
import json
import openpyxl

# Load the workbook
wb = openpyxl.load_workbook('shop_site.xlsx')

comments_of_shops = {}
# Iterate over each sheet
for sheet_name in wb.sheetnames:
    if sheet_name == 'Sheet1':
        continue
    # Select the sheet
    sheet = wb[sheet_name]

    comments = []

    # Get the row values (assuming first row is title)
    titles = [cell.value for cell in sheet[1]]

    # skip if empty
    if not titles:
        continue

    # Iterate over each row starting from 2nd row (assuming first row is title)
    for row_num in range(2, sheet.max_row + 1):
        comment = {}
        # Get the row values
        row_values = [cell.value for cell in sheet[row_num]]
        for i in range(len(titles)):
            comment[titles[i]] = row_values[i]
        comments.append(comment)
    comments_of_shops[sheet_name] = comments

with open('comments_of_shops.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(comments_of_shops, ensure_ascii=False))


In [58]:
import json
# print(open('comments_of_shops.json','r').read())
comments_of_shops = json.loads(open('comments_of_shops.json','r').read())

In [61]:
import re
from tqdm import tqdm
from ollama_chat import chat_reponse
llama3 = 'llama3'
gemma2 = 'gemma2:9b-instruct-q6_K'
gemma2_b = 'gemma2:27b-instruct-q5_K_M'
qwen2 = 'qwen2:7b-instruct-q6_K'

keys = ['value', 'immersive', 'guidance', 'followup', 'content']
# a regex string that matches the first five 1 digit numbers seperated by spaces to these keys: value, immersive, guidance, followup, and content. Then all other charactors that follows
review_re = r'(?P<value>\d)\s(?P<immersive>\d)\s(?P<guidance>\d)\s(?P<followup>\d)\s(?P<content>\d)(?P<reason>.*)'

for shop in comments_of_shops:
    print(shop)
    comments = comments_of_shops[shop]
    for comment in tqdm(comments):
        if comment.get('reason'):
            continue
        text = comment.get('Content')
        if not text:
            continue
        message = \
        """用户将输入来自某沉浸式艺术展的用户评论，请完整地阅读评论并对它进行分析。
        ###
        按照下面的五个方面对他进行打分：
        1. 性价比不高：消费者对于性价比有负面评价，这种评价来源于期望与实际体验之间的不匹配
        2. 沉浸体验很差：消费者对于付费进行沉浸体验本身的轻视、不尊重或不认可
        3. 体验引导不清晰：消费者常因体验的缺乏引导性而产生迷失感，对整体体验感到不满
        4. 后续消费环节/后续服务不充足：消费者在体验结束后仍对体验持续性有更高期待，认为缺少后续环节如周边，纪念品，收集品，信息交流渠道等
        5. 展览内容的负面评价：消费者对于沉浸展览的主题内容有负面评价，认为主题内容难以接受，例如太抽象、无法理解、看不懂等
        
        ###
        每一方面总分为5分，只能打整数分。
        评分时完全基于用户评价的文字内容，每个分数都要有自己参考的评论内容。
        只有评论中出现这一方面的负面评论时，才进行扣分。评价越消极扣分越多。
        分数越低说明这个评价包含的评价越符合这一个方面的描述。

        ###
        如果评论中没有提及某一个方面的相关信息，则这个方面评满分。
        如果需要推测评论者在这个方面的态度，则直接评满分。
        不要进行任何的推测，如果没有相关的信息直接评满分。

        ###
        在进行回复时，请严格按照以下格式：
        <分数一> <分数二> <分数三> <分数四> <分数五>
        <评价理由>

        ###
        不要在回复中添加任何的前缀！
        评分理由中对每一个分数给出分别给出理由。
        """
        review = chat_reponse(text, model=gemma2, system_prompt=message)
        # print(text)
        # print(review['message']['content'])
        try:
            match_dict = re.match(review_re, review['message']['content'], re.DOTALL).groupdict()
        except AttributeError:
            print(review['message']['content'])
            continue
        for key, value in match_dict.items():
            if key == 'reason':
                value = value.strip()
            comment[key] = value

with open('comments_of_shops_with_review.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(comments_of_shops, ensure_ascii=False))

再现山海经·大型沉浸式体验馆


100%|██████████| 2/2 [00:12<00:00,  6.25s/it]


印象莫奈时光映迹艺术展2023-成都站


100%|██████████| 122/122 [06:43<00:00,  3.31s/it]


致敬达芬奇2.0光影艺术体验大展


...

100%|██████████| 975/975 [55:13<00:00,  3.40s/it]


图坦卡蒙·探秘古埃及沉浸展


100%|██████████| 148/148 [07:32<00:00,  3.06s/it]


梵高全沉浸式艺术馆


100%|██████████| 38/38 [01:57<00:00,  3.10s/it]
